In [19]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time

##　进行谓词实例化

In [2]:
# 文件路径
file_path = "prompts/DataFountain/predicates.txt"

# 请求的 URL
url = "https://api.moonshot.cn/v1/files"

headers = {
    'Authorization': 'sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw'
}

# 准备文件数据
files = {'file': open(file_path, 'rb')}

# 发起 POST 请求
response = requests.post(url, headers=headers, files=files)

# 打印响应内容
print(response.status_code)
print(response.text)

200
{"id":"ct03gj3jfih8i2ke32bg","object":"file","bytes":6815,"created_at":1732261964,"filename":"predicates.txt","purpose":"file-extract","status":"ok","status_details":""}


In [20]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

file_object = client.files.create(file=Path("prompts/DataFountain/predicates.txt"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text

In [21]:
print(file_content)

{"content":"任务描述：你将被给予一条 JSON 格式的数据，包含一条规则（rule）的描述，这条规则中包含的谓词（predicates），以及根据规则转换而来的逻辑表达式（logicexpression）。你的任务是针对谓词取不同的具体的值，代入逻辑表达式中，其中取值要满足多样性与复杂性，并将实例化的参数值输出为可解析的 JSON 格式。\n------\n下面是一个例子：\n```json\n{\n    \"rule\": \"安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。\",\n    \"predicates\":\n    [\n        \"指示：Instruction(Leader, Regulation) ::: 表示领导发出的指示和相关规定。\",\n        \"参与：Participate(Inspector, Scene, Task) ::: 表示安全监察专员在事故现场参与某项任务。\",\n        \"赶赴：GoTo(Inspector, Scene) ::: 表示安全监察专员赶赴事故现场。\",\n        \"事故：Accident(Scene) ::: 表示发生在某个场景的事故。\"\n    ],\n    \"logicexpression\":\n    [\n        \"∀s ∀i ∀r (Accident(s) ∧ Instruction(Leader, r) → (GoTo(i, s) ∧ Participate(i, s, EmergencyRescue) ∧ Participate(i, s, Investigation)))\"\n    ]\n}\n```\n###\n这条例子的谓词实例化的参数值，和将实例化参数代入逻辑表达式的输出如下：\n[\n    {\n        \"instance_predicates\": [\n            \"s=“化工厂” — 表示事故场景在化工厂\",\n            \"i=“张三” — 表示安全监察专员为张三\",\n            \"r=“化工事故应急处理规定” — 表示领导发出的指示和相关规定\"\n    

## 读取数据与格式转换

In [5]:
import re

with open('data/DF/answers.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [6]:
print(data[0])

根据提供的规则描述，我们可以将其解析为一阶逻辑公式。以下是规则的逻辑表达式：

```json
{
    "rule": "危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。",
    "predicates": [
        "危险化学品事故：ChemicalAccident(Cause, Injury, PropertyDamage, EnvironmentalPollution) ::: 表示由危险化学品造成的事故，包括人员伤害、财产损失和环境污染。",
        "造成：Cause(Accident, Effect) ::: 表示事故造成了某种效果。",
        "人员伤害：Injury(Person) ::: 表示事故中人员受到伤害。",
        "财产损失：PropertyDamage(Asset) ::: 表示事故中财产受到损失。",
        "环境污染：EnvironmentalPollution(Environment) ::: 表示事故中环境受到污染。",
        "生产：Produce(Chemical) ::: 表示危险化学品的生产过程。",
        "经营：Operate(Chemical) ::: 表示危险化学品的经营过程。",
        "储存：Store(Chemical) ::: 表示危险化学品的储存过程。",
        "运输：Transport(Chemical) ::: 表示危险化学品的运输过程。",
        "使用：Use(Chemical) ::: 表示危险化学品的使用过程。",
        "废弃处置：Dispose(Chemical) ::: 表示危险化学品的废弃处置过程。",
        "矿山开采事故：MiningAccident(GasPoisoning, Explosion, Blasting) ::: 表示矿山开采过程中发生的事故，包括有毒、有害气体中毒、爆炸事故、放炮事故。"
    ],
    "logicexpression": [
        "∀c (Chemi

In [22]:
def extract_json(text):
    # 使用正则表达式匹配并提取 JSON 数据（假设 JSON 数据用 `{` 开头和 `}` 结尾）
    json_data = re.search(r'\{.*\}', text, re.DOTALL)

    # 检查是否找到匹配的 JSON 数据
    if json_data:
        json_string = json_data.group(0)

        # # 将 JSON 数据解析为字典
        # data = json.loads(json_string)
        return json_string
    else:
        print("未找到 JSON 数据。")

In [15]:
extract_json(data[0])

{'rule': '危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。',
 'predicates': ['危险化学品事故：ChemicalAccident(Cause, Injury, PropertyDamage, EnvironmentalPollution) ::: 表示由危险化学品造成的事故，包括人员伤害、财产损失和环境污染。',
  '造成：Cause(Accident, Effect) ::: 表示事故造成了某种效果。',
  '人员伤害：Injury(Person) ::: 表示事故中人员受到伤害。',
  '财产损失：PropertyDamage(Asset) ::: 表示事故中财产受到损失。',
  '环境污染：EnvironmentalPollution(Environment) ::: 表示事故中环境受到污染。',
  '生产：Produce(Chemical) ::: 表示危险化学品的生产过程。',
  '经营：Operate(Chemical) ::: 表示危险化学品的经营过程。',
  '储存：Store(Chemical) ::: 表示危险化学品的储存过程。',
  '运输：Transport(Chemical) ::: 表示危险化学品的运输过程。',
  '使用：Use(Chemical) ::: 表示危险化学品的使用过程。',
  '废弃处置：Dispose(Chemical) ::: 表示危险化学品的废弃处置过程。',
  '矿山开采事故：MiningAccident(GasPoisoning, Explosion, Blasting) ::: 表示矿山开采过程中发生的事故，包括有毒、有害气体中毒、爆炸事故、放炮事故。'],
 'logicexpression': ['∀c (ChemicalAccident(c, i, p, e) ↔ (Produce(c) ∨ Operate(c) ∨ Store(c) ∨ Transport(c) ∨ Use(c) ∨ Dispose(c)) ∧ (Cause(c, i) ∧ Cause(c, p) ∧ Cause(c, e)) ∧ ¬(

## 使用api进行谓词实例化

In [10]:
def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            return completion
        except Exception as e:
            if '429' in str(e):  # 检查异常消息中是否包含429
                print("Rate limit reached. Retrying in 1 second...")
                time.sleep(1)  # 等待1秒钟
                retries += 1
            else:
                raise  # Re-raise the exception if it's not a 429 error
    raise Exception("Max retries reached. API call failed.")

In [11]:
instance_predicates = []

In [17]:
print(file_content)

{"content":"任务描述：你将被给予一条 JSON 格式的数据，包含一条规则（rule）的描述，这条规则中包含的谓词（predicates），以及根据规则转换而来的逻辑表达式（logicexpression）。你的任务是针对谓词取不同的具体的值，代入逻辑表达式中，其中取值要满足多样性与复杂性，并将实例化的参数值输出为可解析的 JSON 格式。\n------\n下面是一个例子：\n```json\n{\n    \"rule\": \"安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。\",\n    \"predicates\":\n    [\n        \"指示：Instruction(Leader, Regulation) ::: 表示领导发出的指示和相关规定。\",\n        \"参与：Participate(Inspector, Scene, Task) ::: 表示安全监察专员在事故现场参与某项任务。\",\n        \"赶赴：GoTo(Inspector, Scene) ::: 表示安全监察专员赶赴事故现场。\",\n        \"事故：Accident(Scene) ::: 表示发生在某个场景的事故。\"\n    ],\n    \"logicexpression\":\n    [\n        \"∀s ∀i ∀r (Accident(s) ∧ Instruction(Leader, r) → (GoTo(i, s) ∧ Participate(i, s, EmergencyRescue) ∧ Participate(i, s, Investigation)))\"\n    ]\n}\n```\n###\n这条例子的谓词实例化的参数值，和将实例化参数代入逻辑表达式的输出如下：\n[\n    {\n        \"instance_predicates\": [\n            \"s=“化工厂” — 表示事故场景在化工厂\",\n            \"i=“张三” — 表示安全监察专员为张三\",\n            \"r=“化工事故应急处理规定” — 表示领导发出的指示和相关规定\"\n    

In [23]:
for index, text in enumerate(data[0: 100], start=0):
    # 替换内容
    replacement = extract_json(text)

    # 使用正则表达式替换 [[RULES]]
    content = re.sub(r'\[\[RULES\]\]', replacement, file_content)

    # # 使用replace
    # content = file_content.replace("[[RULES]]", replacement)

    messages = [
        {
            "role": "system",
            "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。请将内容输出为可解析的json格式。"
        },
        {
            "role": "system",
            "content": content,
        },
        {
            "role": "user",
            "content": "你好，请根据txt内容生成回答。"
        }
    ]

    completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

    print(f"{index}: {text}")
    instance_predicates.append(completion.choices[0].message.content)

0: 根据提供的规则描述，我们可以将其解析为一阶逻辑公式。以下是规则的逻辑表达式：

```json
{
    "rule": "危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。",
    "predicates": [
        "危险化学品事故：ChemicalAccident(Cause, Injury, PropertyDamage, EnvironmentalPollution) ::: 表示由危险化学品造成的事故，包括人员伤害、财产损失和环境污染。",
        "造成：Cause(Accident, Effect) ::: 表示事故造成了某种效果。",
        "人员伤害：Injury(Person) ::: 表示事故中人员受到伤害。",
        "财产损失：PropertyDamage(Asset) ::: 表示事故中财产受到损失。",
        "环境污染：EnvironmentalPollution(Environment) ::: 表示事故中环境受到污染。",
        "生产：Produce(Chemical) ::: 表示危险化学品的生产过程。",
        "经营：Operate(Chemical) ::: 表示危险化学品的经营过程。",
        "储存：Store(Chemical) ::: 表示危险化学品的储存过程。",
        "运输：Transport(Chemical) ::: 表示危险化学品的运输过程。",
        "使用：Use(Chemical) ::: 表示危险化学品的使用过程。",
        "废弃处置：Dispose(Chemical) ::: 表示危险化学品的废弃处置过程。",
        "矿山开采事故：MiningAccident(GasPoisoning, Explosion, Blasting) ::: 表示矿山开采过程中发生的事故，包括有毒、有害气体中毒、爆炸事故、放炮事故。"
    ],
    "logicexpression": [
        "∀c (Ch

TypeError: decoding to str: need a bytes-like object, NoneType found

In [24]:
with open('data/DF/instance_predicates.json', 'w', encoding='utf-8') as f:
    json.dump(instance_predicates, f, ensure_ascii=False, indent=4)

In [32]:
print(instance_predicates[5])

根据您提供的规则数据，以下是谓词实例化的参数值，以及将实例化参数代入逻辑表达式的结果：

```json
[
    {
        "instance_predicates": [
            "d=“调度统计司” — 表示调度统计司负责应急值守",
            "i=“化工厂爆炸事故” — 表示接收的事故信息为化工厂爆炸事故",
            "r=“安全监管总局领导” — 表示报告的安全监管总局领导",
            "l=“事故救援处理工作指导意见” — 表示起草的事故救援处理工作指导意见"
        ],
        "instance_logicexpression": [
            "∀d ∀i ∀r (EmergencyDuty(“调度统计司”) ∧ ReceiveAccidentInfo(“调度统计司”, “化工厂爆炸事故”) ∧ HandleAccidentInfo(“调度统计司”, “化工厂爆炸事故”) ∧ ReportToLeader(“调度统计司”, “化工厂爆炸事故”) ∧ ForwardToOffice(“调度统计司”, “化工厂爆炸事故”) ∧ ForwardToCommandCenter(“调度统计司”, “化工厂爆炸事故”))",
            "∀d ∀i (EmergencyDuty(“调度统计司”) → (∃l “事故救援处理工作指导意见” ∧ DraftGuidance(“调度统计司”, “事故救援处理工作指导意见”)))",
            "∀d (EmergencyDuty(“调度统计司”) → (TrackRescue(“调度统计司”) ∧ ContinueReportRescue(“调度统计司”)))"
        ]
    },
    {
        "instance_predicates": [
            "d=“调度统计司” — 表示调度统计司负责应急值守",
            "i=“矿井塌方事故” — 表示接收的事故信息为矿井塌方事故",
            "r=“安全监管总局领导” — 表示报告的安全监管总局领导",
            "l=“矿井救援行动方案” — 